# 0. Install Required Libraries

In [1]:
%%capture
! pip install google-api-python-client
! pip install pytube
! pip install transformers

# 1. Import Models

In [2]:
from pytube import extract
from googleapiclient.discovery import build
import re
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

#3. Build Google's Youtube API

In [3]:
api_key = "your youtube data v3 api key"
service = build('youtube', 'v3', developerKey=api_key)

#4. Extract **VIDEO ID** from youtube's video url

In [4]:
def get_video_id (url) :
  return extract.video_id(url)

#5. Text cleaning for comments


*   remove hyperlinks
*   remove timestamps



In [5]:
def clean_txt (text) :
  text = text.lower()
  text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)
  return text

#6. Extracts newest 100 comments of given url

In [6]:
def get_comments (video_id) :
  req = service.commentThreads().list(
        part = "snippet",
        videoId = video_id,
        maxResults = 100
    )

  response = req.execute()

  comments = []

  for i in response['items'] :
    cmt = i['snippet']['topLevelComment']['snippet']['textDisplay']
    cmt = clean_txt(cmt)
    comments.append(cmt)

  return comments

#7. Load required Hugging Face tokenizer and models

In [7]:
def load_models () :

  tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
  model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

  return tokenizer, model

In [8]:
%%capture
tokenizer, model = load_models()

#8. Prediction function

In [9]:
def predict (comments) :
  for text in comments :
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    output = torch.argmax(output[0][0])

    if output == 0 :
      print("Negative")
    
    elif output == 1 :
      print("Neutral")
    
    else :
      print("Positive")

#9. Main function

In [10]:
def show (url) :
  url = get_video_id(url)
  comments = get_comments(url)
  predict(comments)

#10. Example

In [ ]:
url = "youtube video url"

show(url)